# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()



This error most likely means that this notebook is not configured to use a GPU.  Change this in Notebook Settings via the command palette (cmd/ctrl-shift-P) or the Edit menu.




InvalidArgumentError: ignored

In [0]:
import tensorflow as tf
import time
import math

import os
import pickle
import skimage as ski
import skimage.io
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

tf.reset_default_graph()

class TFConv2d:
    
    def __init__(self,width=1, height=1, channels=1):
        
        height = 28
        width = 28
        channels = 1
        n_inputs = height * width
        
        self.width = width
        self.height = height
        self.channels = channels
        
        self.conv1_fmaps = 16
        self.conv1_ksize = 5
        self.conv1_stride = 1
        self.conv1_pad = "SAME"
        
        self.conv2_fmaps = 32
        self.conv2_ksize = 5
        self.conv2_stride = 1
        self.conv2_pad = "SAME"
        
        self.pool3_fmaps = self.conv2_fmaps
        self.learning_rate = 0.1

        self.n_fc1 = 512
        self.n_outputs = 10
        
        self.n_inputs = width * height
        
        print(height, width, channels)
        with tf.name_scope("model"):
          self.X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
          self.X_reshaped = tf.reshape(self.X, shape=[-1, height, width, channels])
          self.y = tf.placeholder(tf.int32, shape=[None], name="y")

          self.l2_reg = 0.001
          self.regularizer = tf.contrib.layers.l2_regularizer(self.l2_reg)

          self.conv1 = tf.layers.conv2d(self.X_reshaped, filters= self.conv1_fmaps, kernel_size = self.conv1_ksize
                                  ,strides = self.conv1_stride, padding = self.conv1_pad, kernel_regularizer = self.regularizer,
                                  activation=None, name="my_conv1")
          self.max_pool1 = tf.nn.max_pool(self.conv1, ksize=[1,2,2,1], strides=[1,2,2,1],padding="VALID")

          self.relu1 = tf.nn.relu(self.max_pool1)

          self.conv2 = tf.layers.conv2d(self.relu1, filters=self.conv2_fmaps, kernel_size=self.conv2_ksize,
                                  strides=self.conv2_stride, padding=self.conv2_pad, kernel_regularizer = self.regularizer,
                                  activation=None, name="my_conv2")

          self.max_pool2 = tf.nn.max_pool(self.conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")

          self.relu2 = tf.nn.relu(self.max_pool2)

          self.pool2_flattern = tf.reshape(self.relu2, shape=[-1, self.pool3_fmaps * 7 * 7])

          self.fc1 = tf.layers.dense(self.pool2_flattern, self.n_fc1, activation=tf.nn.relu,kernel_regularizer = self.regularizer,
                                    name="fc")

          self.logits = tf.layers.dense(self.fc1, self.n_outputs, name="output")
          self.Y_proba = tf.nn.softmax(self.logits, name="Y_proba")

          self.xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits,
                                                                    labels=self.y)
          self.loss = tf.reduce_mean(self.xentropy)
          self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
          self.training_op = self.optimizer.minimize(self.loss)

          self.correct = tf.nn.in_top_k(self.logits, self.y, 1)
          self.accuracy = tf.reduce_mean(tf.cast(self.correct, tf.float32))

          self.sess = tf.Session()
          self.init = tf.global_variables_initializer()
          self.saver = tf.train.Saver()
    
    def evaluate(self, X_eval, y_true, my_title="train"):
        print("Usao sam")
        y_pred = np.argmax(self.prediction(X_eval),axis=1)
        cnf_matrix = confusion_matrix(y_true, y_pred)
        cnn_recall = recall_score(y_true, y_pred, average='micro')
        cnn_precision = precision_score(y_true, y_pred, average='micro')
        cnn_acc = accuracy_score(y_true, y_pred)
        print(my_title)
        print(cnf_matrix)
        return  cnn_acc, cnn_precision, cnn_recall
    
    def prediction(self, X_eval):
        sess = self.sess
#             self.saver.restore(sess, "./batch_conv2d_para.ckpt")
#         sess.run(self.init)
        return sess.run(self.Y_proba, feed_dict={self.X: X_eval})
    
    def draw_conv_filters(self, weights):
        w = weights.copy()
        num_filters = w.shape[3]
        num_channels = w.shape[2]
        k = w.shape[0]
        assert w.shape[0] == w.shape[1]
        w = w.reshape(k, k, num_channels, num_filters)
        w -= w.min()
        w /= w.max()
        border = 1
        cols = 8
        rows = math.ceil(num_filters / cols)
        width = cols * k + (cols-1) * border
        height = rows * k + (rows-1) * border
        img = np.zeros([np.int(height), np.int(width), np.int(num_channels)])
        for i in range(np.int(num_filters)):
            r = int(i / cols) * (k + border)
            c = int(i % cols) * (k + border)
            img[r:r+k,c:c+k,:] = w[:,:,:,i]
#         img.astype(np.uint8)
#         ski.io.imshow(img)
#         ski.io.show()
        print("Shape is ", img.shape)
        imgplot = plt.imshow(img.squeeze())
        plt.show()
  
    
    def visual_con(self):
        conv1_var = tf.contrib.framework.get_variables('my_conv1/kernel:0')[0]
        conv1_weights = conv1_var.eval(session=self.sess)
        self.draw_conv_filters(conv1_weights)
    
    
    def top_3(self, X_train, y_train, niter=1, batch_size = 1):
        sess = self.sess
        
        n, N= X_train.shape
        s = np.arange(n)
        
        ako = True
        
        
        max_loss1 = 0
        max_loss2 = 0
        max_loss3 = 0
        X1 = None
        X2 = None
        X3 = None
        y1 = None
        y2 = None
        y3 = None
        
        if ako:
            sess.run(self.init)
    
            for i in range(niter):
            
                n_batches = n // batch_size
                np.random.shuffle(s)
                X_train = X_train[s]
                y_train = y_train[s]
                
                for j in range(n_batches):
                    X_batch = X_train[j*batch_size: (j+1)*batch_size]
                    y_batch = y_train[j*batch_size: (j+1)*batch_size]
                    
                    loss = sess.run([ self.loss], feed_dict={self.X: X_batch, self.y: y_batch})
                    
                    if(loss > max_loss1 and i==(niter-1)):
                        X1 = X_batch
                        y1 = y_batch
                        max_loss1 = loss
                    elif(loss > max_loss2  and i==(niter-1)):
                        X2 = X_batch
                        y2 = y_batch
                        max_loss2 = loss
                    elif(loss > max_loss3  and i==(niter-1)):
                        X3 = X_batch
                        y3 = y_batch
                        max_loss3 = loss

            print("First loss is",X1)
            print("Predictio is", self.prediction(X1))
            print("Real class is", y1)
            print("First loss is",X2)
            print("Predictio is", self.prediction(X2))
            print("Real class is", y2)
            print("First loss is",X3)
            print("Predictio is", self.prediction(X3))
            print("Real class is", y3)
    
    
  
    def batch_train(self, X, y, niter=10000, batch_size = 50):
        sess = self.sess
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0, test_size=0.1)
        
        n, N= X_train.shape
        s = np.arange(n)
        
        ako = True
        acc_train = []
        recall_train = []
        precision_train = []
        acc_valid = []
        recall_valid = []
        precision_valid = []
        
        max_loss1 = 0
        max_loss2 = 0
        max_loss3 = 0
        X1 = None
        X2 = None
        X3 = None
        y1 = None
        y2 = None
        y3 = None
        
        loss_s = []
        if ako:
            sess.run(self.init)
    
            for i in range(niter):
                loss_uk = 0
                n_batches = n // batch_size
                np.random.shuffle(s)
                X_train = X_train[s]
                y_train = y_train[s]
                
                for j in range(n_batches):
                    X_batch = X_train[j*batch_size: (j+1)*batch_size]
                    y_batch = y_train[j*batch_size: (j+1)*batch_size]
                    
                    _, loss = sess.run([self.training_op, self.loss], feed_dict={self.X: X_batch, self.y: y_batch})
                    loss_uk = loss_uk +loss
                    if(loss > max_loss1 and i==(niter-1)):
                        X1 = X_batch
                        y1 = y_batch
                        max_loss1 = loss 
                    elif(loss > max_loss2  and i==(niter-1)):
                        X2 = X_batch
                        y2 = y_batch
                        max_loss2 = loss
                    elif(loss > max_loss3  and i==(niter-1)):
                        X3 = X_batch
                        y3 = y_batch
                        max_loss3 = loss

                loss_s +=[loss_uk]
                cnn_acc_train, cnn_precision_train, cnn_recall_train= self.evaluate(X_train, y_train)
                acc_train += [cnn_acc_train]
                recall_train += [cnn_recall_train]
                precision_train += [cnn_precision_train]
                cnn_acc_valid, cnn_precision_valid, cnn_recall_valid= self.evaluate(X_valid, y_valid, my_title="valid")
                acc_valid += [cnn_acc_valid]
                recall_valid += [cnn_recall_valid]
                precision_valid += [cnn_precision_valid]
            self.visual_con()
            print("First loss is",X1)
            print("Predictio is", self.prediction(X1))
            print("Real class is", y1)
            print("First loss is",X2)
            print("Predictio is", self.prediction(X2))
            print("Real class is", y2)
            print("First loss is",X3)
            print("Predictio is", self.prediction(X3))
            print("Real class is", y3)
        fig, axes = plt.subplots(4,1,figsize=(20, 10))
        axes[0].plot(np.linspace(1.0, niter,num=niter), acc_train, label='accuracy train')
        axes[0].legend(loc='best')
        axes[0].set_xlabel('number of iteration')



        axes[1].plot(np.linspace(1.0, niter,num=niter), recall_train, label= 'recall train')


        axes[1].legend(loc='best')
        axes[1].set_xlabel('number of iteration')
        
        axes[2].plot(np.linspace(1.0, niter,num=niter), cnn_precision_train, label='precision train')
        axes[2].legend(loc='best')
        axes[2].set_xlabel('number of iteration')
        
        axes[0].plot(np.linspace(1.0, niter,num=niter), acc_valid, label='accuracy valid')
        



        axes[1].plot(np.linspace(1.0, niter,num=niter), recall_valid, label= 'recall valid')
        
        axes[2].plot(np.linspace(1.0, niter,num=niter), cnn_precision_valid, label='precision valid')
        
        axes[3].plot(np.linspace(1.0, niter,num=niter), loss_s, label='loss')
        axes[3].legend(loc='best')
        axes[3].set_xlabel('number of iteration')

        plt.show()
    def pokreni_me(self):
      n_epochs = 10
      batch_size = 100

      with tf.Session() as sess:
          sess.run(self.init)
          for epoch in range(n_epochs):
              for iteration in range(mnist.train.num_examples // batch_size):
                  X_batch, y_batch = mnist.train.next_batch(batch_size)
                  sess.run(self.training_op, feed_dict={self.X: X_batch, self.y: y_batch})
              acc_train = accuracy.eval(feed_dict={self.X: X_batch, self.y: y_batch})
              acc_test = accuracy.eval(feed_dict={self.X: mnist.test.images, self.y: mnist.test.labels})
              print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

              save_path = saver.save(sess, "./my_mnist_model")
if __name__ == '__main__':

    mnist = input_data.read_data_sets("/tmp/data/")
    my_CNN = TFConv2d(height=28, width=28)
    my_CNN.batch_train(mnist.train.images, mnist.train.labels, niter=2, batch_size = 50)
    my_CNN.top_3(mnist.train.images, mnist.train.labels)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
(28, 28, 1)
Usao sam


In [0]:
my_CNN = TFConv2d(height=28, width=28)
my_CNN.pokreni_me()

(28, 28, 1)


InvalidArgumentError: ignored

In [0]:
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 10

tf.reset_default_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
n_epochs = 10
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

        save_path = saver.save(sess, "./my_mnist_model")

(0, 'Train accuracy:', 0.96, 'Test accuracy:', 0.9823)
(1, 'Train accuracy:', 0.97, 'Test accuracy:', 0.9862)
(2, 'Train accuracy:', 0.99, 'Test accuracy:', 0.9859)
(3, 'Train accuracy:', 1.0, 'Test accuracy:', 0.987)
(4, 'Train accuracy:', 1.0, 'Test accuracy:', 0.9897)
(5, 'Train accuracy:', 1.0, 'Test accuracy:', 0.9891)


KeyboardInterrupt: ignored

In [0]:
height = 28
width = 28
channels = 1
n_inputs = height * width
        

        
conv1_fmaps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 32
conv2_ksize = 5
conv2_stride = 1
conv2_pad = "SAME"
        
pool3_fmaps = conv2_fmaps
learning_rate = 0.001

n_fc1 = 512
n_outputs = 10
        
n_inputs = width * height
   
  
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
y = tf.placeholder(tf.int32, shape=[None], name="y")

l2_reg = 0.001
regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
               
conv1 = tf.layers.conv2d(X_reshaped, filters= conv1_fmaps, kernel_size = conv1_ksize
                                ,strides = conv1_stride, padding = conv1_pad, kernel_regularizer = regularizer,
                                activation=None, name="my_conv1")
max_pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1],padding="VALID")
        
relu1 = tf.nn.relu(max_pool1)
        
conv2 = tf.layers.conv2d(relu1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                                strides=conv2_stride, padding=conv2_pad, kernel_regularizer = regularizer,
                                activation=None, name="my_conv2")



# with tf.name_scope("pool3"):
#     pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
#     pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])

max_pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")
        
relu2 = tf.nn.relu(max_pool2)
        
pool2_flattern = tf.reshape(relu2, shape=[-1, pool3_fmaps * 7 * 7])
        
fc1 = tf.layers.dense(pool2_flattern, n_fc1, activation=tf.nn.relu,kernel_regularizer = regularizer,
                                  name="fc")
        
logits = tf.layers.dense(fc1, n_outputs, name="output")
Y_proba = tf.nn.softmax(logits, name="Y_proba")
        
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                                  labels=y)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
        
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [0]:
n_epochs = 10
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

        save_path = saver.save(sess, "./my_mnist_model")

(0, 'Train accuracy:', 0.99, 'Test accuracy:', 0.9797)
(1, 'Train accuracy:', 0.98, 'Test accuracy:', 0.9882)
(2, 'Train accuracy:', 0.99, 'Test accuracy:', 0.9863)
(3, 'Train accuracy:', 0.99, 'Test accuracy:', 0.9878)
(4, 'Train accuracy:', 1.0, 'Test accuracy:', 0.9921)
(5, 'Train accuracy:', 1.0, 'Test accuracy:', 0.9877)
(6, 'Train accuracy:', 1.0, 'Test accuracy:', 0.9917)
(7, 'Train accuracy:', 1.0, 'Test accuracy:', 0.9926)
(8, 'Train accuracy:', 1.0, 'Test accuracy:', 0.9911)
(9, 'Train accuracy:', 1.0, 'Test accuracy:', 0.99)


In [48]:
import tensorflow as tf
import time
import math

import os
import pickle
import skimage as ski
import skimage.io
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

tf.reset_default_graph()


DATA_DIR = "./"
SAVE_DIR = "./"


def shuffle_data(data_x, data_y):
  indices = np.arange(data_x.shape[0])
  np.random.shuffle(indices)
  shuffled_data_x = np.ascontiguousarray(data_x[indices])
  shuffled_data_y = np.ascontiguousarray(data_y[indices])
  return shuffled_data_x, shuffled_data_y

def unpickle(file):
  fo = open(file, 'rb')
  dict = pickle.load(fo, encoding='latin1')
  fo.close()
  return dict


def draw_conv_filters(epoch, step, weights, save_dir):
  w = weights.copy()
  num_filters = w.shape[3]
  num_channels = w.shape[2]
  k = w.shape[0]
  assert w.shape[0] == w.shape[1]
  w = w.reshape(k, k, num_channels, num_filters)
  w -= w.min()
  w /= w.max()
  border = 1
  cols = 8
  rows = math.ceil(num_filters / cols)
  width = cols * k + (cols-1) * border
  height = rows * k + (rows-1) * border
  img = np.zeros([height, width, num_channels])
  #img = np.zeros([height, width])
  for i in range(num_filters):
    r = int(i / cols) * (k + border)
    c = int(i % cols) * (k + border)
    img[r:r+k,c:c+k,:] = w[:,:,:,i]
  filename = 'epoch_%02d_step_%06d.png' % (epoch, step)
  if(img.shape[2]==1):
      zipped = np.dstack((img, img, img))
  else:
      zipped = img
  print(img.shape)
  ski.io.imsave(os.path.join(save_dir, filename), zipped)


  
def plot_training_progress(save_dir, data):
  fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,8))

  linewidth = 2
  legend_size = 10
  train_color = 'm'
  val_color = 'c'

  num_points = len(data['train_loss'])
  x_data = np.linspace(1, num_points, num_points)
  ax1.set_title('Cross-entropy loss')
  ax1.plot(x_data, data['train_loss'], marker='o', color=train_color,
           linewidth=linewidth, linestyle='-', label='train')
  ax1.plot(x_data, data['valid_loss'], marker='o', color=val_color,
           linewidth=linewidth, linestyle='-', label='validation')
  ax1.legend(loc='upper right', fontsize=legend_size)
  ax2.set_title('Average class accuracy')
  ax2.plot(x_data, data['train_acc'], marker='o', color=train_color,
           linewidth=linewidth, linestyle='-', label='train')
  ax2.plot(x_data, data['valid_acc'], marker='o', color=val_color,
           linewidth=linewidth, linestyle='-', label='validation')
  ax2.legend(loc='upper left', fontsize=legend_size)
  ax3.set_title('Learning rate')
  ax3.plot(x_data, data['lr'], marker='o', color=train_color,
           linewidth=linewidth, linestyle='-', label='learning_rate')
  ax3.legend(loc='upper left', fontsize=legend_size)

  save_path = os.path.join(save_dir, 'training_plot.pdf')
  print('Plotting in: ', save_path)
  plt.savefig(save_path)

  
def class_to_onehot(Y):
  Yoh=np.zeros((len(Y),max(Y)+1))
  Yoh[range(len(Y)),Y] = 1
  return Yoh




class TFConv2d_CIFAR:
    
    def __init__(self,width=1, height=1, channels=1):
        
        
        self.width = width
        self.height = height
        self.channels = channels
        
        self.conv1_fmaps = 16
        self.conv1_ksize = 5
        self.conv1_stride = 1
        self.conv1_pad = "SAME"
        
        self.conv2_fmaps = 32
        self.conv2_ksize = 5
        self.conv2_stride = 1
        self.conv2_pad = "SAME"
        
        pool3_fmaps = conv2_fmaps
        learning_rate = 0.001

        n_fc1 = 256
        n_fc2 = 128
        n_outputs = 10
        
        self.X = tf.palceholder(tf.float32, shape=[None, self.width * self.height], name="X")
        self.X_reshaped = tf.reshape(self.X, shape=[-1, self.height, self.width, self.channels])
        self.y = tf.placeholder(tf.int32, shape=[None], name="y")

        l2_reg = 0.001
        regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
               
        self.conv1 = tf.layers.conv2d(self.X_reshaped, filters= conv1_fmaps, kernel_size = conv1_ksize
                                ,strides = conv1_stride, padding = conv1_pad, kernel_regularizer = None,
                                activation=tf.nn.relu, name="conv1")
        self.max_pool1 = tf.nn.max_pool(self.conv1, ksize=[1,3,3,1], strides=[1,2,2,1],padding="VALID")
        
        
        self.conv2 = tf.layers.conv2d(self.max_pool1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                                strides=conv2_stride, padding=conv2_pad, kernel_regularizer = None,
                                activation=tf.nn.relu, name="conv2")
        
        self.max_pool2 = tf.nn.max_pool(self.conv2, ksize=[1,3,3,1], strides=[1,2,2,1], padding="VALID")
        
        self.pool2_flattern = tf.reshape(self.max_pool2, shape=[-1, pool3_fmaps *8*8])
        
        self.fc1 = tf.layers.dense(self.pool2_flattern, n_fc1, activation=tf.nn.relu,kernel_regularizer = None,
                                  name="fc1")
        
        self.fc2 = tf.layers.dense(self.fc1, n_fc2, activation=tf.nn.relu, kernel_regularizer = None,
                                  name="fc2")
        
        self.logits = tf.layers.dense(self.fc2, n_outputs, name="output")
        self.Y_proba = tf.nn.softmax(self.logits, name="Y_proba")
        
        self.xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits,
                                                                  labels=y_train)
        self.loss = tf.reduce_mean(self.xentropy)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        self.training_op = self.optimizer.minimize(self.loss)
        
        self.correct = tf.nn.in_top_k(self.logits, self.y, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct, tf.float32))
        
        self.init = tf.global_variables_initializer()
        self.saver = tf.train.Saver()
        self.sess = tf.Session()
    
    def evaluate(self, X_eval, y_true, my_title="train"):
        print("Usao sam")
        y_pred = np.argmax(self.prediction(X_eval),axis=1)
        cnf_matrix = confusion_matrix(y_true, y_pred)
        cnn_recall = recall_score(y_true, y_pred, average='micro')
        cnn_precision = precision_score(y_true, y_pred, average='micro')
        cnn_acc = accuracy_score(y_true, y_pred)
        print(my_title)
        print(cnf_matrix)
        return  cnn_acc, cnn_precision, cnn_recall
    
    def prediction(self, X_eval):
        sess = self.sess
#             self.saver.restore(sess, "./batch_conv2d_para.ckpt")
#         sess.run(self.init)
        return sess.run(self.Y_proba, feed_dict={self.X: X_eval})
    
    def draw_conv_filters(self, weights):
        w = weights.copy()
        num_filters = w.shape[3]
        num_channels = w.shape[2]
        k = w.shape[0]
        assert w.shape[0] == w.shape[1]
        w = w.reshape(k, k, num_channels, num_filters)
        w -= w.min()
        w /= w.max()
        border = 1
        cols = 8
        rows = math.ceil(num_filters / cols)
        width = cols * k + (cols-1) * border
        height = rows * k + (rows-1) * border
        img = np.zeros([np.int(height), np.int(width), np.int(num_channels)])
        for i in range(np.int(num_filters)):
            r = int(i / cols) * (k + border)
            c = int(i % cols) * (k + border)
            img[r:r+k,c:c+k,:] = w[:,:,:,i]
#         img.astype(np.uint8)
#         ski.io.imshow(img)
#         ski.io.show()
        print("Shape is ", img.shape)
        imgplot = plt.imshow(img.squeeze())
        plt.show()
  
    
    def visual_con(self):
        conv1_var = tf.contrib.framework.get_variables('my_conv1/kernel:0')[0]
        conv1_weights = conv1_var.eval(session=self.sess)
        self.draw_conv_filters(conv1_weights)
  
  
    def batch_train(self, X, y, niter=10000, batch_size = 50):
        sess = self.sess
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0, test_size=0.1)
        
        n, N= X_train.shape
        s = np.arange(n)
        
        ako = True
        acc_train = []
        recall_train = []
        precision_train = []
        acc_valid = []
        recall_valid = []
        precision_valid = []
        
        max_loss1 = 0
        max_loss2 = 0
        max_loss3 = 0
        X1 = None
        X2 = None
        X3 = None
        y1 = None
        y2 = None
        y3 = None
        
        if ako:
            sess.run(self.init)
    
            for i in range(niter):
            
                n_batches = n // batch_size
                np.random.shuffle(s)
                X_train = X_train[s]
                y_train = y_train[s]
                
                for j in range(n_batches):
                    X_batch = X_train[j*batch_size: (j+1)*batch_size]
                    y_batch = y_train[j*batch_size: (j+1)*batch_size]
                    
                    _, loss = sess.run([self.training_op, self.loss], feed_dict={self.X: X_batch, self.y: y_batch})
                    
                    if(loss > max_loss1):
                        X1 = X_batch
                        y1 = y_batch
                        max_loss1 = loss
                    elif(loss > max_loss2):
                        X2 = X_batch
                        y2 = y_batch
                        max_loss2 = loss
                    elif(loss > max_loss3):
                        X3 = X_batch
                        y3 = y_batch
                        max_loss3 = loss

#                 cnn_acc_train, cnn_precision_train, cnn_recall_train= self.evaluate(X_train, y_train)
#                 acc_train += [cnn_acc_train]
#                 recall_train += [cnn_recall_train]
#                 precision_train += [cnn_precision_train]
                cnn_acc_valid, cnn_precision_valid, cnn_recall_valid= self.evaluate(X_valid, y_valid, my_title="valid")
                acc_valid += [cnn_acc_valid]
                recall_valid += [cnn_recall_valid]
                precision_valid += [cnn_precision_valid]
            self.visual_con()
            print("First loss is",X1)
            print("Predictio is", self.prediction(X1))
            print("Real class is", y1)
            print("First loss is",X2)
            print("Predictio is", self.prediction(X2))
            print("Real class is", y2)
            print("First loss is",X3)
            print("Predictio is", self.prediction(X3))
            print("Real class is", y3)
#         fig, axes = plt.subplots(6,1,figsize=(20, 10))
#         axes[0].plot(np.linspace(1.0, niter,num=niter), acc_train, label='accuracy train')
#         axes[0].legend(loc='best')
#         axes[0].set_xlabel('number of iteration')



#         axes[1].plot(np.linspace(1.0, niter,num=niter), recall_train, label= 'recall train')


#         axes[1].legend(loc='best')
#         axes[1].set_xlabel('number of iteration')
        
#         axes[2].plot(np.linspace(1.0, niter,num=niter), cnn_precision_train, label='precision train')
#         axes[2].legend(loc='best')
#         axes[2].set_xlabel('number of iteration')
        
#         axes[3].plot(np.linspace(1.0, niter,num=niter), acc_valid, label='accuracy valid')
#         axes[3].legend(loc='best')
#         axes[3].set_xlabel('number of iteration')



#         axes[4].plot(np.linspace(1.0, niter,num=niter), recall_valid, label= 'recall valid')


#         axes[4].legend(loc='best')
#         axes[4].set_xlabel('number of iteration')
        
#         axes[5].plot(np.linspace(1.0, niter,num=niter), cnn_precision_valid, label='precision valid')
#         axes[5].legend(loc='best')
#         axes[5].set_xlabel('number of iteration')

#         plt.show()
    
    
   
if __name__=="__main__":
    np.random.seed(100)
    tf.set_random_seed(100)
    tf.reset_default_graph()

    config = {}
    config['max_epochs'] = 8
    config['batch_size'] = 50
    config['save_dir'] = SAVE_DIR
    config['lr_policy'] = {1:{'lr':1e-1}, 3:{'lr':1e-2}, 5:{'lr':1e-3}, 7:{'lr':1e-4}}
    
    img_height, img_width = 32, 32
    num_channels = 3
    num_classes = 10
    
    train_x = np.ndarray((0, img_height * img_width * num_channels), dtype=np.float32)
    train_y = []
    for i in range(1, 6):
      subset = unpickle(os.path.join(DATA_DIR, 'data_batch_%d' % i))
      train_x = np.vstack((train_x, subset['data']))
      train_y += subset['labels']
    train_x = train_x.reshape((-1, num_channels, img_height, img_width)).transpose(0,2,3,1)
    train_y = np.array(train_y, dtype=np.int32)
    
    subset = unpickle(os.path.join(DATA_DIR, 'test_batch'))
    test_x = subset['data'].reshape((-1, num_channels, img_height, img_width)).transpose(0,2,3,1).astype(np.float32)
    test_y = np.array(subset['labels'], dtype=np.int32)
    
    valid_size = 5000
    train_x, train_y = shuffle_data(train_x, train_y)
    valid_x = train_x[:valid_size, ...]
    valid_y = train_y[:valid_size, ...]
    train_x = train_x[valid_size:, ...]
    train_y = train_y[valid_size:, ...]
    data_mean = train_x.mean((0,1,2))
    data_std = train_x.std((0,1,2))
    
    train_x = (train_x - data_mean) / data_std
    valid_x = (valid_x - data_mean) / data_std
    test_x = (test_x - data_mean) / data_std

    learning_rate = tf.placeholder(tf.float32, shape=[])
    X = tf.placeholder(tf.float32,[None,img_height,img_width,num_channels])
    Y = tf.placeholder(tf.float32,[None,num_classes])

#     train_y = class_to_onehot(train_y)
#     valid_y = class_to_onehot(valid_y)
#     test_y = class_to_onehot(test_y)
    
    my_cifra_cnn = TFConv2d_CIFAR(width=img_width, height=img_height, channels=num_channels)
    my_cifra_nn.batch_train(train_x,train_y, niter=2)
    
#     logits, loss = build_model(X, Y, num_classes)
    
#     trainer = tf.train.GradientDescentOptimizer(learning_rate)
#     train_op = trainer.minimize(loss)
    
#     sess = tf.Session()

#     run_ops = [logits, loss, train_op]
#     logits,loss = train(train_x, train_y, valid_x, valid_y, run_ops, config, sess)
#     # here it dies
#     run_ops = [logits, loss]
#     evaluate("Test", test_x, test_y, config, run_ops, sess)
        

KeyError: ignored

In [37]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("./") if isfile(join("./", f))]
print(onlyfiles)

['batch_conv2d_para.ckpt.meta', 'cifar-10-python', 'batch_conv2d_para.ckpt.data-00000-of-00001', 'checkpoint', 'data_batch_1', '.rnd', 'batch_conv2d_para.ckpt.index', 'cifar-10-python.tar']


In [34]:
import urllib2
import StringIO
import gzip

baseURL = "https://www.cs.toronto.edu/~kriz/"
filename = "cifar-10-python.tar.gz"
outFilePath = "./"

response = urllib2.urlopen(baseURL + filename)
compressedFile = StringIO.StringIO()
compressedFile.write(response.read())
#
# Set the file's current position to the beginning
# of the file so that gzip.GzipFile can read
# its contents from the top.
#
compressedFile.seek(0)

decompressedFile = gzip.GzipFile(fileobj=compressedFile, mode='rb')

with open(outFilePath, 'w') as outfile:
    outfile.write(decompressedFile.read())

IOError: ignored

In [0]:
import requests
import os
import shutil

global dump

def download_file():
    global dump
    url = "https://drive.google.com/open?id=1Boewt6HNcdLFpUx_AHe12TXTNGY7ZS0Z"
    file = requests.get(url, stream=True)
    dump = file.raw

def save_file():
    global dump
    location = os.path.abspath("./")
    with open("test_batch", 'wb') as location:
        shutil.copyfileobj(dump, location)
    del dump
    
download_file()
save_file()

In [46]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving batches.meta to batches.meta
Saving data_batch_1 to data_batch_1 (1)
Saving data_batch_2 to data_batch_2 (1)
Saving data_batch_3 to data_batch_3 (1)
Saving data_batch_4 to data_batch_4 (1)
Saving data_batch_5 to data_batch_5 (1)
Saving readme.html to readme.html
Saving test_batch to test_batch (1)
User uploaded file "data_batch_1" with length 31035704 bytes
User uploaded file "data_batch_2" with length 31035320 bytes
User uploaded file "data_batch_3" with length 31035999 bytes
User uploaded file "data_batch_4" with length 31035696 bytes
User uploaded file "batches.meta" with length 158 bytes
User uploaded file "test_batch" with length 31035526 bytes
User uploaded file "readme.html" with length 88 bytes
User uploaded file "data_batch_5" with length 31035623 bytes
